In [1]:
import random
import numpy as np

# graphs
import matplotlib.pyplot as plt
import networkx as nx
from networkx.generators.random_graphs import erdos_renyi_graph, complete_graph
from networkx.generators import circulant_graph

In [2]:
import numpy as np
from typing import List


class BaseSmoothOracle(object):
    """
    Base class for implementation of oracles.
    """

    def func(self, x):
        """
        Computes the value of function at point x.
        """
        raise NotImplementedError('Func oracle is not implemented.')

    def grad(self, x):
        """
        Computes the gradient at point x.
        """
        raise NotImplementedError('Grad oracle is not implemented.')

    def hess(self, x):
        """
        Computes the Hessian matrix at point x.
        """
        raise NotImplementedError('Hessian oracle is not implemented.')

    def func_directional(self, x, d, alpha):
        """
        Computes phi(alpha) = f(x + alpha*d).
        """
        return np.squeeze(self.func(x + alpha * d))

    def grad_directional(self, x, d, alpha):
        """
        Computes phi'(alpha) = (f(x + alpha*d))'_{alpha}
        """
        return np.squeeze(self.grad(x + alpha * d).dot(d))


class OracleLinearComb(BaseSmoothOracle):
    """
    Implements linear combination of several oracles with given coefficients.
    Resulting oracle = sum_{m=1}^M coefs[m] * oracles[m].

    Parameters
    ----------
    oracles: List[BaseSmoothOracle]

    coefs: List[float]
    """

    def __init__(self, oracles: List[BaseSmoothOracle], coefs: List[float]):
        if len(oracles) != len(coefs):
            raise ValueError("Numbers of oracles and coefs should be equal!")
        self.oracles = oracles
        self.coefs = coefs

    def func(self, x: np.ndarray) -> float:
        res = 0
        for oracle, coef in zip(self.oracles, self.coefs):
            res += oracle.func(x) * coef
        return res

    def grad(self, x: np.ndarray) -> np.ndarray:
        res = self.oracles[0].grad(x) * self.coefs[0]
        for oracle, coef in zip(self.oracles[1:], self.coefs[1:]):
            res += oracle.grad(x) * coef
        return res


import numpy as np
from scipy.special import expit


class LinearRegressionL2Oracle(BaseSmoothOracle):
    """
    Linear regression oracle with L2 regularization.
    1/2 \|Ax - b\|_2^2 + regcoef * \|x\|_2^2.

    Parameters
    ----------
    A: np.ndarray
        Feature matrix

    b: np.ndarray
        Target vector
    """

    def __init__(self, A: np.ndarray, b: np.ndarray, regcoef: float, reduction: str = "mean"):
        self.A = A
        self.b = b
        self.regcoef = regcoef
        self.den = A.shape[0] if reduction == "mean" else 1

    def func(self, x: np.ndarray):
        residual = self.A.dot(x) - self.b
        return 0.5 * residual.dot(residual) / self.den + 0.5 * self.regcoef * x.dot(x)

    def grad(self, x: np.ndarray):
        residual = self.A.dot(x) - self.b
        return self.A.T.dot(residual) / self.den + self.regcoef * x


In [3]:
import numpy as np
import numpy.linalg as npla
from collections import defaultdict
from datetime import datetime
from typing import Optional


class BaseMethod(object):
    """
    Base class for minimization methods.

    Parameters
    ----------
    oracle: BaseSmoothOracle
        Oracle corresponding to the objective function.

    x_0: np.ndarray
        Initial guess.

    stopping_criteria: Optional[str]
        Str specifying stopping criteria. Supported values:
        "grad_rel": terminate if ||f'(x_k)||^2 / ||f'(x_0)||^2 <= eps
        "grad_abs": terminate if ||f'(x_k)||^2 <= eps
        "func_abs": terminate if f(x_k) <= eps (implicitly assumes that f* = 0)

    trace: bool
        If True, saves the history of the method during its iterations.
    """

    def __init__(self, oracle, x_0, stopping_criteria, trace):
        self.oracle = oracle
        self.x_0 = x_0.copy()
        self.trace = trace
        if stopping_criteria == 'grad_rel':
            self.stopping_criteria = self.stopping_criteria_grad_relative
        elif stopping_criteria == 'grad_abs':
            self.stopping_criteria = self.stopping_criteria_grad_absolute
        elif stopping_criteria == 'func_abs':
            self.stopping_criteria = self.stopping_criteria_func_absolute
        elif stopping_criteria == None:
            self.stopping_criteria = self.stopping_criteria_none
        else:
            raise ValueError('Unknown stopping criteria type: "{}"' \
                             .format(stopping_criteria))

    def run(self, max_iter=10, max_time=1200):
        """
        Run method for a maximum of max_iter iterations and max_time seconds.

        Parameters
        ----------
        max_iter: int
            Maximum number of iterations

        max_time: int
            Maximum running time
        """

        if not hasattr(self, 'hist'):
            self.hist = defaultdict(list)
        if not hasattr(self, 'time'):
            self.time = 0.

        self._absolute_time = datetime.now()
        try:
            for iter_count in range(max_iter):
                if self.time > max_time:
                    break
                if self.trace:
                    self._update_history()
                self.step()

                if self.stopping_criteria():
                    break
        except KeyboardInterrupt:
            print('Run interrupted at iter #{}'.format(iter_count))

        self.hist['x_star'] = self.x_k.copy()

    def _update_history(self):
        """
        Updates self.hist: saves time, function values and gradient norm.
        """

        now = datetime.now()
        self.time += (now - self._absolute_time).total_seconds()
        self._absolute_time = now
        self.hist['func'].append(self.oracle.func(self.x_k))
        self.hist['time'].append(self.time)
        if not hasattr(self, 'grad_k'):
            self.grad_k = self.oracle.grad(self.x_k)
        self.hist['grad_norm'].append(npla.norm(self.grad_k))

    def step(self):
        raise NotImplementedError('step() not implemented!')

    def stopping_criteria_grad_relative(self):
        return npla.norm(self.grad_k) ** 2 <= self.tolerance * self.grad_norm_0 ** 2

    def stopping_criteria_grad_absolute(self):
        return npla.norm(self.grad_k) ** 2 <= self.tolerance

    def stopping_criteria_func_absolute(self):
        return self.oracle.func(self.x_k) < self.tolerance

    def stopping_criteria_none(self):
        return False


class BaseDecentralizedMethod(object):
    def __init__(self, oracle_list, x_0: np.ndarray, logger: "LoggerDecentralized"):
        self.oracle_list = oracle_list
        self.x = x_0.copy()
        self.logger = logger

    def run(self, max_iter: int, max_time: Optional[float] = None):
        if self.logger is not None:
            self.logger.start(self)
        if max_time is None:
            max_time = +np.inf
        if not hasattr(self, 'time'):
            self.time = 0.

        self._absolute_time = datetime.now()
        for iter_count in range(max_iter):
            if self.time > max_time:
                break
            self._update_time()
            if self.logger is not None:
                self.logger.step(self)
            self.step()

        if self.logger is not None:
            self.logger.step(self)
            self.logger.end(self)

    def _update_time(self):
        now = datetime.now()
        self.time += (now - self._absolute_time).total_seconds()
        self._absolute_time = now

    def step(self):
        raise NotImplementedError('step() not implemented!')

    def func_list(self, x: np.ndarray) -> np.ndarray:
        return np.array([self.oracle_list[i].func(x[i]) for i in range(len(self.oracle_list))])

    def grad_list(self, x: np.ndarray) -> np.ndarray:
        return np.vstack([self.oracle_list[i].grad(x[i]) for i in range(len(self.oracle_list))])

    
class LoggerDecentralized(object):
    def __init__(self, x_true: Optional[np.ndarray] = None):
        self.func_avg = []
        self.time = []
        self.sq_dist_to_con = []
        self.x_true = None
        if x_true is not None:
            self.x_true = x_true.copy()
            self.sq_dist_avg_to_opt = []

    def start(self, method: BaseDecentralizedMethod):
        pass

    def end(self, method: BaseDecentralizedMethod):
        pass

    def step(self, method: BaseDecentralizedMethod):
        self.func_avg.append(np.mean(method.func_list(method.x)))
        self.time.append(method.time)
        self.sq_dist_to_con.append(
            ((method.x - method.x.mean(axis=0)) ** 2).sum() / method.x.shape[0]
        )
        if self.x_true is not None:
            self.sq_dist_avg_to_opt.append(((method.x.mean(axis=0) - self.x_true) ** 2).sum())


In [4]:
class DAPDG(BaseDecentralizedMethod):
    
    """
    oracle_list: list of size M of oracles for every node
    x_0: matrix of shape [M, N], must be if Range A^T
    B: matrix of affine constraints of shape [P, N]
    W: laplasian matrix of graph of shape [M, M]
    logger: haven't figured it out yet
    eta, alpha, beta, theta, sigma, gamma: parameters
    """
    def __init__(
            self,
            oracle_list: List[BaseSmoothOracle],
            x_0: np.ndarray,
            B: np.ndarray,
            W: np.ndarray,
            logger: LoggerDecentralized,
            eta: float = 1.0,
            alpha: float = 0.75,
            beta: float = 0.01,
            theta: float = 0.01,
            sigma: float = 0.01,
            gamma: float = 0.75
    ):
        super().__init__(oracle_list, x_0, logger)
        
        assert eta > 0 and alpha > 0 and beta > 0
        assert 0 < theta <= 1 and 0 < sigma <= 1
        
        self.eta = eta
        self.alpha = alpha
        self.beta = beta
        self.theta = theta
        self.sigma = sigma
        self.gamma = gamma  # parameter for matrix A
        
        self.B = B
        self.W = W
        
        M = W.shape[0]
        N = B.shape[1]
        self.N, self.M = N, M
        
        assert W.shape == (M, M)
        assert self.x.shape == (M, N)
        
        self.x_f = np.zeros((M, N))
        
        bold_B = np.kron(np.ones((M, M)), B)
        bold_W = np.kron(W, np.ones((N, N)))
        self.A = np.vstack([bold_B, gamma * bold_W])
        
        # x = np.random.rand(N * M)
        # self.x = A.T.dot(x)


    def step(self):
        A = self.A
        x = self.x.reshape(M * N)
        x_f = self.x_f.reshape(M * N)
        x_g = self.theta * x + (1 - self.theta) * x_f
        
        grad_F_x_g = self.grad_list(x_g.reshape(M, N)).reshape(N * M)  # np.array[N * M]
        
        x_new = x + self.eta * self.alpha * (x_g - x) - \
            self.eta * (self.beta * A.T.dot(A.dot(x)) - grad_F_x_g)
        
        x_f = x_g + self.sigma * (x_new - x)
        
        self.x = x_new.reshape(M, N)
        self.x_f = x_f.reshape(M, N)
        
        
    """
    W: laplasian matrix of graph of shape [M, M]
    """
    def set_new_graph(self, W):
        assert W.shape == (self.M, self.M)
        bold_B = np.kron(np.ones((self.M, self.M)), self.B)
        bold_W = np.kron(W, np.ones((self.N, self.N)))
        self.A = np.vstack([bold_B, self.gamma * bold_W])

In [5]:
N = 5  # dimension of space 
M = 3  # count of nodes
P = 4  # dimension of constraints

In [6]:
x = np.random.rand(M, N)
B = np.random.rand(P, N)
B[-1] = B[:-1].sum(axis=0)  # to create non-trivial kernel
G = circulant_graph(M, [1])
W = nx.laplacian_matrix(G).A  # may be better not to convert into numpy array

oracle_list = [LinearRegressionL2Oracle(np.random.rand(N, N), np.random.rand(N), 1) for _ in range(M)]
logger = LoggerDecentralized()

<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


In [7]:
opt = DAPDG(oracle_list, x, B, W, logger)
opt.run(max_iter=1000)

In [8]:
opt.x

array([[ 35572820.74461409, -42485569.49523211, -38579025.73353057,
        -59508813.25168604,  -9682133.42163074],
       [ 21618621.8754817 , -52465228.31537428, -47952446.50594428,
        -18623748.57477611, -28178427.71944028],
       [ 29290273.76489308, -96327153.27602203, -23341182.46902759,
        -31181389.64041551,  17406214.30634402]])

In [9]:
import itertools
from tqdm import tqdm

x = np.random.rand(M,N)
min_x = 1000
best_success = -1
success_params = []
for eta, alpha, beta, theta, sigma, delta in tqdm(itertools.product(np.linspace(0.01, 1, 5), np.linspace(0.01, 1, 5), np.linspace(0.01, 1, 5),
                                                             np.linspace(0.01, 1, 5), np.linspace(0.01, 1, 5), np.linspace(0.01, 1, 5))):
    opt = DAPDG(oracle_list, x, B, W, logger, eta, alpha, beta, theta, sigma, delta)
    opt.run(max_iter=100)
    
    res = (opt.x ** 2).sum()
    if res < min_x:
        success = [res, eta, alpha, beta, theta, sigma, delta]
        mix_x = res

3502it [00:29, 119.93it/s]/home/kongor/.local/lib/python3.10/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/tmp/ipykernel_67582/342121956.py:172: RuntimeWarning: overflow encountered in square
  ((method.x - method.x.mean(axis=0)) ** 2).sum() / method.x.shape[0]
/tmp/ipykernel_67582/2068259480.py:13: RuntimeWarning: overflow encountered in square
  res = (opt.x ** 2).sum()
15625it [02:12, 117.54it/s]


In [10]:
success

[31.467909414988306, 1.0, 0.7525, 0.01, 0.01, 0.01, 0.7525]